In [ ]:
type Tableau
    c::Array{Float64}
    b::Array{Float64}
    A::Array{Float64}
    obj::Float64
    basicas::Array{Int16}
end



In [ ]:
#introducimos datos
data=Tableau([2500 4000 0 0 0], [200; 100; 750], [1 0 1 0 0; 0 1 0 1 0; 3 5 0 0 1],0.0,[3 4 5])

In [ ]:
function selectPivote(data)
    #println("costosBasicos: ",data.c)
    colIn = indmax(data.c) #devuelve posicion con mayor costo reducido
    if data.c[colIn] <= 0.0
        println("=====================================================")
        println("Solucion Optima")
        return -1,-1
    end
    #println("\tEntra En Base:",colIn)
    maxVal = typemax(Float64)
    rowOut = -1
    for i in 1:size(data.A,1)
        if(data.A[i,colIn]>0.0) && (maxVal > data.b[i]/data.A[i,colIn])
            maxVal = data.b[i]/data.A[i,colIn]
            rowOut=i
        end
    end
    if rowOut == -1
        println("=====================================================")
        println("Problema no acotado") 
        return -1,-1
    end
    #println("entra columna ",colIn," sale fila ",rowOut)
    colIn,rowOut
end

function IteracionSimplex!(data)
    variableIn,variableOut = selectPivote(data)
    if variableIn == -1
        return true
    end
    #cambiar variable de la base
    data.basicas[variableOut]=variableIn
    pivotValue=data.A[variableOut,variableIn]
    #println("pivotValue: ",pivotValue)
    #cambiar matriz A y vector b
    data.A[variableOut,:]=data.A[variableOut,:]./pivotValue
    data.b[variableOut]=data.b[variableOut]/pivotValue
    for rows in 1:size(data.A,1)
        if rows == variableOut
            continue
        end
        if data.A[rows,variableOut] != 0.0
            multiplier = data.A[rows,variableOut]
            data.A[rows,:]=data.A[rows,:]-multiplier*data.A[variableOut,:]
            data.b[rows]=data.b[rows]-multiplier*data.b[variableOut]
        end
    end
    #actualizar obj
    data.obj += data.c[variableIn]*data.b[variableOut]
    data.c = data.c - data.c[variableIn]*data.A[variableOut,:]
    #println("obj: ",data.obj)
    #println("vectorC: ",data.c)
    #println("matriz A:")
    #println(data.A)
    #println("vector B:")
    #println(data.A)
    return false
end

function printTableau(data)
    println("=====================================================")
    println("obj: ",data.obj," size ",size(data.c,2), " basicas ",size(data.A,1))
    for i in 1:size(data.c,2)
        if i>1 
            print(" + ")
        end
        print(data.c[i],"*x_",i," ")
    end
    print("\n")
    for i in 1:size(data.A,1)
        print(data.basicas[i]," ")
        for j in 1:size(data.A,2)
            if j>1 
                print(" + ")
            end
            @printf("%5.2f*x_%d ",data.A[i,j],j)
        end
        print(" = ",data.b[i],"\n")
    end
end

In [ ]:
printTableau(data)
while true
    if IteracionSimplex!(data)
        break
    else
        printTableau(data)
    end
end